In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

### Data

In [5]:
data = { "artist" : ["The Beatles", "Pink Floyd", "Metallica","Cairokee", "ACDC",
                     "The Doors","Poets of The Fall"],
       "country" : ["UK", "UK", "US","Egypt","US","US","Finland"],
       "plays" : [150,10000,500,200,250,1000,250],
       "genre" : ["rock","rock","metal", "rock","rock","rock","rock"]}

df = pd.DataFrame(data)

### Filtering

In [6]:
uk_artists = df[df['country']=='UK']
display(uk_artists)

out = df[(df['genre']=='rock') & (df['plays']>=200)]
display(out)

out = df[~((df.country=='US') & (df.plays >= 500))]
display(out)

out = df[df["country"].isin(["US","UK"]) ]
display(out)

out = df[((df['country']=='UK') & (df['plays'] > 200) ) \
         | ((df['country']!='UK') &( df['plays'] > 100))]
display(out)

,artist,country,plays,genre
0,The Beatles,UK,150,rock
1,Pink Floyd,UK,10000,rock


,artist,country,plays,genre
1,Pink Floyd,UK,10000,rock
3,Cairokee,Egypt,200,rock
4,ACDC,US,250,rock
5,The Doors,US,1000,rock
6,Poets of The Fall,Finland,250,rock


,artist,country,plays,genre
0,The Beatles,UK,150,rock
1,Pink Floyd,UK,10000,rock
3,Cairokee,Egypt,200,rock
4,ACDC,US,250,rock
6,Poets of The Fall,Finland,250,rock


,artist,country,plays,genre
0,The Beatles,UK,150,rock
1,Pink Floyd,UK,10000,rock
2,Metallica,US,500,metal
4,ACDC,US,250,rock
5,The Doors,US,1000,rock


,artist,country,plays,genre
1,Pink Floyd,UK,10000,rock
2,Metallica,US,500,metal
3,Cairokee,Egypt,200,rock
4,ACDC,US,250,rock
5,The Doors,US,1000,rock
6,Poets of The Fall,Finland,250,rock


### String filters

In [7]:
out = df[df['artist'].str.startswith('The')]
out = df[df.artist.str.contains('The')]

out = out[out['country'].isin(['UK', 'Finland'])]
display(out)

filtered = df[(~df.country.isin(['UK', 'Finland'])) | (df.plays >= 10000)]
print(filtered.artist.values)

,artist,country,plays,genre
0,The Beatles,UK,150,rock
6,Poets of The Fall,Finland,250,rock


['Pink Floyd' 'Metallica' 'Cairokee' 'ACDC' 'The Doors']


In [8]:
%%file music.csv
artist,country,plays,genre,fans
The Beatles,UK,150,rock,50
Pink Floyd,UK,10000,rock,1500
Metallica,US,500,metal,50
Cairokee,Egypt,200,rock,10
ACDC,US,250,rock,20
The Doors,US,1000,rock,80
Poets of The Fall,Finland,250,rock,10
Megadeth,US,300,metal,20
Iron Maiden,UK,20000,metal,3500
Judas Priest,UK,5000,metal,1000

Overwriting music.csv


In [9]:
df = pd.read_csv('music.csv')

### groupby

In [46]:
print(df.groupby('country').plays)   # series
print(df.groupby('country').plays.sum().index)
print(df.groupby('country').plays.sum().to_dict())

Index(['Egypt', 'Finland', 'UK', 'US'], dtype='object', name='country')
{'Egypt': 200, 'Finland': 250, 'UK': 35150, 'US': 2050}


In [47]:
df.groupby(['country', 'genre']).plays.sum().to_dict()

{('Egypt', 'rock'): 200,
 ('Finland', 'rock'): 250,
 ('UK', 'metal'): 25000,
 ('UK', 'rock'): 10150,
 ('US', 'metal'): 800,
 ('US', 'rock'): 1250}

### reset_index

In [55]:
non_reset = df.groupby('country').plays.sum()
print(non_reset,"\n")


reset = df.groupby('country').plays.sum().reset_index()
print(reset,"\n")

print((reset[reset.plays>1000].country.values))

country
Egypt        200
Finland      250
UK         35150
US          2050
Name: plays, dtype: int64 

   country  plays
0    Egypt    200
1  Finland    250
2       UK  35150
3       US   2050 

['UK' 'US']


In [63]:
reset = df.groupby('country').plays.sum()
filtered = reset[reset > 1000]
print((filtered.index))
print(list(filtered.index))

Index(['UK', 'US'], dtype='object', name='country')
['UK', 'US']


### apply

Your music analyst would like to know the ratio of plays/fans (plays per fan) to see how dedicated listeners are to artists. This comes with a twist: it should be done per nation, not per artist.

In [65]:
df.groupby('country').apply(lambda x: x.plays.sum() / x.fans.sum())


country
Egypt      20.000000
Finland    25.000000
UK          5.809917
US         12.058824
dtype: float64

In [66]:
df.groupby('country').apply(lambda x: x.plays.sum() / x.fans.sum()).to_dict()

{'Egypt': 20.0,
 'Finland': 25.0,
 'UK': 5.809917355371901,
 'US': 12.058823529411764}

### Aggregate

Your music analyst is interested in knowing multiple statistics at once, grouped by country.These are as follows:

- Sum of plays
- Average of plays
- Maximum fans from all artists in the country

In [85]:
grp = df.groupby('country').agg({'plays': ['sum', 'mean'], 'fans': ['max']})
display(grp)

grp.columns = ['_'.join(col) for col in grp.columns.values]
print(grp.columns,"\n")

res = grp.to_dict(orient='index')
print(res)

plays          fans
           sum    mean   max
country                     
Egypt      200   200.0    10
Finland    250   250.0    10
UK       35150  8787.5  3500
US        2050   512.5    80

Index(['plays_sum', 'plays_mean', 'fans_max'], dtype='object') 

{'Egypt': {'plays_sum': 200, 'plays_mean': 200.0, 'fans_max': 10}, 'Finland': {'plays_sum': 250, 'plays_mean': 250.0, 'fans_max': 10}, 'UK': {'plays_sum': 35150, 'plays_mean': 8787.5, 'fans_max': 3500}, 'US': {'plays_sum': 2050, 'plays_mean': 512.5, 'fans_max': 80}}


### Other concepts : Explore 
Passing `as_index=False` while applying a groupby function: When aggregating and applying a reset_index function afterwards, the grouped by column will not be the index of the resultant DataFrame.

**Applying filters on groups using .filter() with a lambda function**: This would allow more concise filters, such as keeping groups with only a count of records greater than a certain value

### Travel dataset

In [90]:
df = pd.read_csv('dates.csv')

df['date'] = df['date'].map(lambda x: pd.to_datetime(x))
df['month'] = df['date'].dt.month

len(df[df['month']==9])



4

In [91]:
df['date'] = pd.to_datetime(df['date'])

start = pd.to_datetime('2019-01-01')
end = pd.to_datetime('2020-01-01')

filtered = df[(df.date >= start) & (df.date < end)]
len(filtered)

44

Your music analyst wants to generate a list of dates to set the concert schedule of a new band, starting 1 January 2019, up-to and including 31 December 2019, all separated by exactly 7 days.

In [96]:
date_list = ([x.strftime("%Y-%m-%d") for x in \
     pd.date_range(start='2019-01-01', end='2019-12-31', freq='7D')])

# date_list


Question : Your music analyst would like to know how many concerts were held in each month present in the dataset.


Pandas has a very handy function for upsampling/downsampling time series: resample(<freq>). The correct freq here is MS, which refers to “Month Start”. It will generate groups by months, identified in the index by the start date of every month.

The solution starts by setting the index to be the date column, and then applying the resampling operation (which is close to a groupby operation); this groups the data by the month. You simply want the size of each group.

The last step is to format the index of the resampled data to match the YYYY-MM expected output, and generate the dict representation.

In [104]:
df = pd.read_csv("dates.csv")
df['date'] = pd.to_datetime(df['date'])

display(df)

resampled = df.set_index('date').resample('MS').size()   # MS : Month Start

print(resampled)

resampled.index = resampled.index.map(lambda x: x.strftime("%Y-%m"))
print(resampled.index)


print(resampled.to_dict())




,date
0,2020-04-21 17:52:20
1,2019-08-15 04:32:38
2,2019-04-12 06:39:37
3,2020-10-04 06:05:16
4,2020-07-21 22:19:35
...,...
95,2019-12-05 22:39:29
96,2020-11-23 18:02:26
97,2019-06-22 05:01:15
98,2020-08-25 21:38:58


date
2019-01-01    3
2019-02-01    3
2019-03-01    1
2019-04-01    4
2019-05-01    3
2019-06-01    5
2019-07-01    2
2019-08-01    7
2019-09-01    2
2019-10-01    6
2019-11-01    2
2019-12-01    6
2020-01-01    2
2020-02-01    4
2020-03-01    6
2020-04-01    5
2020-05-01    2
2020-06-01    5
2020-07-01    8
2020-08-01    5
2020-09-01    2
2020-10-01    5
2020-11-01    3
2020-12-01    4
2021-01-01    2
2021-02-01    3
Freq: MS, dtype: int64
Index(['2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
       '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
       '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
       '2021-01', '2021-02'],
      dtype='object', name='date')
{'2019-01': 3, '2019-02': 3, '2019-03': 1, '2019-04': 4, '2019-05': 3, '2019-06': 5, '2019-07': 2, '2019-08': 7, '2019-09': 2, '2019-10': 6, '2019-11': 2, '2019-12': 6, '2020-01': 2, '2020-02': 4, '2

### Other concepts
Working with time information other than dates
Adding/subtracting dates using timedelta, which can help you shift dates by a specific range e.g., by adding 1 month to an existing value

### Apply/Map 

```python
df['my_col'].map(lambda x: my_magic_fn(x))
```

Your music analyst would like to know the continent of origin of each band. Can you help them with that?

In [125]:
df = pd.read_csv('music.csv')
continents = {'UK': 'Europe', 'US': 'North America', 'Egypt': 'Africa', 'Finland': 'Europe'}

df ["continent"] = df["country"].map(lambda x : continents[x])
df[["artist" , "continent"]].set_index('artist').to_dict()

{'continent': {'The Beatles': 'Europe',
  'Pink Floyd': 'Europe',
  'Metallica': 'North America',
  'Cairokee': 'Africa',
  'ACDC': 'North America',
  'The Doors': 'North America',
  'Poets of The Fall': 'Europe',
  'Megadeth': 'North America',
  'Iron Maiden': 'Europe',
  'Judas Priest': 'Europe'}}

In [127]:
df = pd.read_csv('music.csv')
continents = {'UK': 'Europe', 'US': 'North America', 'Egypt': 'Africa', 'Finland': 'Europe'}
df['continent'] = df['country'].map(continents)
df.set_index('artist')['continent'].to_dict()

{'The Beatles': 'Europe',
 'Pink Floyd': 'Europe',
 'Metallica': 'North America',
 'Cairokee': 'Africa',
 'ACDC': 'North America',
 'The Doors': 'North America',
 'Poets of The Fall': 'Europe',
 'Megadeth': 'North America',
 'Iron Maiden': 'Europe',
 'Judas Priest': 'Europe'}

### band initials
Your music analyst would like to assign nicknames to each band based on the first two letters of their name. Can you help them with that?

In [137]:
df = pd.read_csv('music.csv')
df['initials'] = df['artist'].map(lambda x : x[0:2])
df.set_index("artist").initials.to_dict()

{'The Beatles': 'Th',
 'Pink Floyd': 'Pi',
 'Metallica': 'Me',
 'Cairokee': 'Ca',
 'ACDC': 'AC',
 'The Doors': 'Th',
 'Poets of The Fall': 'Po',
 'Megadeth': 'Me',
 'Iron Maiden': 'Ir',
 'Judas Priest': 'Ju'}

### Find popularity of bands

**Since the function isn’t applied to a single column, you use `.apply, with axis=1` to indicate this will be applied to every row (where every row will be defined as x as per the lambda function).**



In [139]:
def is_popular(plays, fans):
    if plays > 1000 or fans > 50:
        return 'very popular'
    elif plays >= 500:
        return 'popular'
    else:
        return 'not popular'
    


df = pd.read_csv('music.csv')
df['popular'] = df.apply(lambda x: is_popular(x.plays, x.fans), axis=1)
df.set_index('artist')['popular'].to_dict()



{'The Beatles': 'not popular',
 'Pink Floyd': 'very popular',
 'Metallica': 'popular',
 'Cairokee': 'not popular',
 'ACDC': 'not popular',
 'The Doors': 'very popular',
 'Poets of The Fall': 'not popular',
 'Megadeth': 'not popular',
 'Iron Maiden': 'very popular',
 'Judas Priest': 'very popular'}

In [140]:
%%file sales.csv
Band,US Sales,UK Sales,Egypt Sales
The Beatles,10k,20k,5k
Black Sabbath,3k,15k,3k

Writing sales.csv


The first step into solving this problem is removing the k suffix, and multiplying the integer value by 1000. This will convert the sales value from thousands of sales.


After cleaning the dataset, it’s important to effectively find the maximum value in each column to get the max per market. To achieve this, you have to apply the max function with the crucial setting of axis=0.

In [141]:
df = pd.read_csv('sales.csv').set_index('Band')

replaced = df.applymap(lambda x: int(x.replace('k', '')) * 1000)
mx = replaced.apply(lambda x: max(x), axis=0)
mx.to_dict()


{'US Sales': 10000, 'UK Sales': 20000, 'Egypt Sales': 5000}

### Basic Merge 

In [142]:
%%file countries.csv
country_id,name
1,UK
2,US
3,Egypt
4,Finland

Writing countries.csv


In [161]:
%%file music_2.csv
artist,country,plays,genre,fans
The Beatles,1,150,rock,50
Pink Floyd,1,10000,rock,1500
Metallica,2,500,metal,50
Cairokee,3,200,rock,10
ACDC,2,250,rock,20
The Doors,2,1000,rock,80
Poets of The Fall,4,250,rock,10
Megadeth,2,300,metal,20
Iron Maiden,1,20000,metal,3500
Judas Priest,1,5000,metal,1000
Leprous,5,1000,metal,500
ABBA,5,2000,pop,100
Bryan Adams,6,2500,pop,200
Rush,6,3000,rock,500

Overwriting music_2.csv


In [162]:
df = pd.read_csv('music_2.csv')
countries = pd.read_csv('countries.csv')

merged = df.merge(countries, left_on='country', right_on='country_id')

merged.groupby('name').plays.sum().to_dict()



{'Egypt': 200, 'Finland': 250, 'UK': 35150, 'US': 2050}

### Merge with Missing Values

In [187]:
df = pd.read_csv('music_2.csv')
countries = pd.read_csv('countries.csv')
merged = pd.merge(df, countries, how='left', left_on='country', right_on='country_id')
merged[merged.name.isnull()].plays.sum()


8500

You can also use a one-liner to solve this problem, without merging at all. What we do here is to filter the music DataFrame by the country column, keeping only the rows where this column is not in the unique values of the countries DataFrame.

In [204]:
df = pd.read_csv('music_2.csv')

list_contries = countries["country_id"].unique()
print(list_contries)

df = df[~df["country"].isin(list_contries)]
display(df)

# onle liner
print(df[~df["country"].isin(list_contries)].plays.sum())

[1 2 3 4]


,artist,country,plays,genre,fans
10,Leprous,5,1000,metal,500
11,ABBA,5,2000,pop,100
12,Bryan Adams,6,2500,pop,200
13,Rush,6,3000,rock,500


8500
